# setup

In [1]:
%%capture
!pip install git+https://github.com/EleutherAI/sae.git

In [2]:
%%capture
!pip install datasets

In [3]:
from sparsify.config import SaeConfig
from sparsify.utils import decoder_impl
from sparsify import Sae

import gc
import pickle
import numpy as np
import matplotlib.pyplot as plt
import json
from fnmatch import fnmatch
from pathlib import Path
from typing import NamedTuple, Optional, Callable, Union, List, Tuple
# from jaxtyping import Float, Int

import einops
import torch
from torch import Tensor, nn
from huggingface_hub import snapshot_download
from natsort import natsorted
from safetensors.torch import load_model, save_model

device = "cuda" if torch.cuda.is_available() else "cpu"

## corr fns

In [4]:
def normalize_byChunks(actv_tensor, chunk_size=10000): # chunk_size: Number of rows per chunk
    mean_A = actv_tensor.mean(dim=0, keepdim=True)
    std_A = actv_tensor.std(dim=0, keepdim=True)

    num_chunks = actv_tensor.shape[0] // chunk_size

    normalized_A = np.zeros_like(actv_tensor.cpu())  # Preallocate the normalized matrix
    # normalized_A = actv_tensor.new_zeros(actv_tensor.size())

    for i in range(num_chunks):
        # print (i, num_chunks)
        start_index = i * chunk_size
        end_index = start_index + chunk_size
        chunk = actv_tensor[start_index:end_index]
        normalized_A[start_index:end_index] = (chunk - mean_A) / (std_A + 1e-8)

    # Handle any remaining rows if the data size is not perfectly divisible by chunk_size
    if actv_tensor.shape[0] % chunk_size != 0:
        start_index = num_chunks * chunk_size
        chunk = actv_tensor[start_index:]
        normalized_A[start_index:] = (chunk - mean_A) / (std_A + 1e-8)

    return torch.tensor(normalized_A)

def batched_correlation(reshaped_activations_A, reshaped_activations_B, batch_size=100):
    # Ensure tensors are on GPU
    # if torch.cuda.is_available():
    #     reshaped_activations_A = reshaped_activations_A.to('cuda')
    #     reshaped_activations_B = reshaped_activations_B.to('cuda')

    normalized_A = normalize_byChunks(reshaped_activations_A, chunk_size=10000)
    normalized_B = normalize_byChunks(reshaped_activations_B, chunk_size=10000)

    if torch.cuda.is_available():
        normalized_A = normalized_A.to('cuda')
        normalized_B = normalized_B.to('cuda')

    num_batches = (normalized_B.shape[1] + batch_size - 1) // batch_size
    max_values = []
    max_indices = []

    for batch in range(num_batches):
        start = batch * batch_size
        # if start % 5000 == 0:
        #     print(start)
        end = min(start + batch_size, normalized_B.shape[1])

        batch_corr_matrix = torch.matmul(normalized_A.t(), normalized_B[:, start:end]) / normalized_A.shape[0]
        max_val, max_idx = batch_corr_matrix.max(dim=0)
        max_values.append(max_val)
        max_indices.append(max_idx)  # Adjust indices for the batch offset

        del batch_corr_matrix
        torch.cuda.empty_cache()

    # return torch.cat(max_indices), torch.cat(max_values)
    return torch.cat(max_indices).cpu().numpy(), torch.cat(max_values).cpu().numpy()

# load model

In [5]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained("EleutherAI/pythia-70m")
tokenizer.pad_token = tokenizer.eos_token
model = AutoModelForCausalLM.from_pretrained("EleutherAI/pythia-70m").to('cuda' if torch.cuda.is_available() else 'cpu')

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
The `GPTNeoXSdpaAttention` class is deprecated in favor of simply modifying the `config._attn_implementation`attribute of the `GPTNeoXAttention` class! It will be removed in v4.48


In [6]:
model_2 = AutoModelForCausalLM.from_pretrained("EleutherAI/pythia-160m").to('cuda' if torch.cuda.is_available() else 'cpu')

# get data

In [7]:
from datasets import load_dataset
dataset = load_dataset("Skylion007/openwebtext", split="train", streaming=True)

In [8]:
def get_next_batch(dataset, batch_size=100, max_length=100):
    batch = []
    dataset_iter = iter(dataset)
    for _ in range(batch_size):
        try:
            sample = next(dataset_iter)
            batch.append(sample['text'])
        except StopIteration:
            break

    # Tokenize the batch
    inputs = tokenizer(batch, return_tensors="pt", padding=True, truncation=True, max_length=max_length)

    return batch, inputs

In [9]:
batch_size = 400
max_length = 400
batch, inputs = get_next_batch(dataset, batch_size=batch_size, max_length=max_length)

In [10]:
inputs['input_ids'].shape  # (batchnum, maxseqlen)

torch.Size([400, 400])

# get llm actvs

In [24]:
from torch.utils.data import DataLoader, TensorDataset

def get_llm_actvs_batch(model, inputs, layerID, batch_size=100, maxseqlen=300):
    accumulated_outputs = None
    dataset = TensorDataset(inputs['input_ids'], inputs['attention_mask'])
    loader = DataLoader(dataset, batch_size=32, shuffle=False)

    for batch in loader:
        input_ids, attention_mask = batch

        batch_inputs = {'input_ids': input_ids.to(model.device), 'attention_mask': attention_mask.to(model.device)}
        with torch.no_grad():
            outputs = model(**batch_inputs, output_hidden_states=True)
            if accumulated_outputs is None:
                accumulated_outputs = outputs.hidden_states[layerID]
            else:
                accumulated_outputs = torch.cat((accumulated_outputs, outputs.hidden_states[layerID]), dim= 0)

        del batch_inputs, outputs
        torch.cuda.empty_cache()
        gc.collect()

    return accumulated_outputs

In [25]:
outputs_by_layer_1 = {}
for layer_id in range(1, 6, 2): # step = 2
    print("Model A Layer: " + str(layer_id))
    with torch.inference_mode():
        outputs_by_layer_1[layer_id] = get_llm_actvs_batch(model, inputs, layer_id)

Model A Layer: 1
Model A Layer: 3
Model A Layer: 5


In [26]:
outputs_by_layer_2 = {}
for layer_id in range(1, 12, 2): # step = 2
    print("Model B Layer: " + str(layer_id))
    with torch.inference_mode():
        outputs_by_layer_2[layer_id] = get_llm_actvs_batch(model_2, inputs, layer_id)

Model B Layer: 1
Model B Layer: 3
Model B Layer: 5
Model B Layer: 7
Model B Layer: 9
Model B Layer: 11


# get sae actvs

In [27]:
def get_weights_and_acts_byLayer_batched(name, layer_id, layer_outputs, batch_size=32):
    """
    Process the sae activations in batches to avoid OOM errors.

    Args:
        name (str): The model name to load from the hub.
        layer_id (int): The layer index to process.
        outputs (Namespace): The model outputs containing hidden_states.
        batch_size (int): The number of samples per batch.

    Returns:
        weight_matrix_np (numpy.ndarray): The decoder weights.
        reshaped_activations_A (torch.Tensor): The pre-activation outputs reshaped.
        orig (torch.Tensor): The original batched pre-activations.
    """
    device = "cuda" if torch.cuda.is_available() else "cpu"
    hookpoint = "layers." + str(layer_id)

    # Load the sae model on the appropriate device.
    sae = Sae.load_from_hub(name, hookpoint=hookpoint, device=device)
    weight_matrix_np = sae.W_dec.cpu().detach().numpy()

    # Init vars to store the hidden states for the given layer.
    num_samples = layer_outputs.size(0)
    pre_act_batches = []

    # Process the activations in batches.
    for start in range(0, num_samples, batch_size):
        end = start + batch_size
        batch = layer_outputs[start:end].to(device)
        with torch.inference_mode():
            batch_pre_acts = sae.pre_acts(batch)
        pre_act_batches.append(batch_pre_acts.cpu())

        # Free up GPU memory.
        del batch, batch_pre_acts
        torch.cuda.empty_cache()
        gc.collect()

    # Concatenate the processed batches.
    orig = torch.cat(pre_act_batches, dim=0)

    first_dim_reshaped = orig.shape[0] * orig.shape[1]
    reshaped_activations_A = orig.reshape(first_dim_reshaped, orig.shape[-1]).cpu()

    return weight_matrix_np, reshaped_activations_A, orig

In [28]:
name = "EleutherAI/sae-pythia-70m-32k"

layer_id = 3
layer_outputs = outputs_by_layer_1[layer_id]
weight_matrix_np, reshaped_activations_A, feature_acts_model_A = get_weights_and_acts_byLayer_batched(name, layer_id, layer_outputs, batch_size=32)

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

In [29]:
name = "EleutherAI/sae-pythia-160m-32k"

layer_id = 5
layer_outputs = outputs_by_layer_2[layer_id]
weight_matrix_2, reshaped_activations_B, feature_acts_model_B = get_weights_and_acts_byLayer_batched(name, layer_id, layer_outputs, batch_size=32)

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

# get llm and saes in one fn

In [11]:
from torch.utils.data import DataLoader, TensorDataset

def get_sae_actvs(model, sae_name, inputs, layer_id, batch_size=32):
    """
    Process the sae activations in batches to avoid OOM errors.

    Args:
        model (torch.nn.Module): The model to process.
        sae_name (str): The SAE model name to load from the hub.
        layer_id (int): The layer index to process.
        batch_size (int): The number of samples per batch.

    Returns:
        weight_matrix_np (numpy.ndarray): The decoder weights.
        reshaped_activations_A (torch.Tensor): The pre-activation outputs reshaped.
        orig (torch.Tensor): The original batched pre-activations.
    """
    device = "cuda" if torch.cuda.is_available() else "cpu"

    LLM_outputs = None
    dataset = TensorDataset(inputs['input_ids'], inputs['attention_mask'])
    loader = DataLoader(dataset, batch_size=batch_size, shuffle=False)

    for batch in loader:
        input_ids, attention_mask = batch

        batch_inputs = {'input_ids': input_ids.to(model.device), 'attention_mask': attention_mask.to(model.device)}
        with torch.no_grad():
            outputs = model(**batch_inputs, output_hidden_states=True)
            if LLM_outputs is None:
                LLM_outputs = outputs.hidden_states[layer_id]
            else:
                LLM_outputs = torch.cat((LLM_outputs, outputs.hidden_states[layer_id]), dim= 0)

        del batch_inputs, outputs
        torch.cuda.empty_cache()
        gc.collect()

    ### Get SAE actvs ###
    hookpoint = "layers." + str(layer_id)

    # Load the sae model on the appropriate device.
    sae = Sae.load_from_hub(sae_name, hookpoint=hookpoint, device=device)
    weight_matrix_np = sae.W_dec.cpu().detach().numpy()

    # Init vars to store the hidden states for the given layer.
    num_samples = LLM_outputs.size(0)
    pre_act_batches = []

    # Process the activations in batches.
    for start in range(0, num_samples, batch_size):
        end = start + batch_size
        batch = LLM_outputs[start:end].to(device)
        with torch.inference_mode():
            batch_pre_acts = sae.pre_acts(batch)
        pre_act_batches.append(batch_pre_acts.cpu())

        # Free up GPU memory.
        del batch, batch_pre_acts
        torch.cuda.empty_cache()
        gc.collect()

    del LLM_outputs
    torch.cuda.empty_cache()
    gc.collect()

    # Concatenate the processed batches.
    orig_actvs = torch.cat(pre_act_batches, dim=0)

    first_dim_reshaped = orig_actvs.shape[0] * orig_actvs.shape[1]
    reshaped_activations_A = orig_actvs.reshape(first_dim_reshaped, orig_actvs.shape[-1]).cpu()

    return weight_matrix_np, reshaped_activations_A, orig_actvs

In [12]:
sae_name = "EleutherAI/sae-pythia-70m-32k"
# sae_actvs_byLayer = {}
layer_id = 3
weight_matrix_1, reshaped_activations_A, feature_acts_model_A =  get_sae_actvs(model, sae_name, inputs, layer_id, batch_size=32)

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

In [13]:
sae_name = "EleutherAI/sae-pythia-160m-32k"
# sae_actvs_byLayer = {}
layer_id = 5
weight_matrix_2, reshaped_activations_B, feature_acts_model_B = get_sae_actvs(model_2, sae_name, inputs, layer_id, batch_size=32)

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

# get corrs

In [ ]:
max_corr_inds, max_corr_vals = batched_correlation(reshaped_activations_A, reshaped_activations_B)

In [ ]:
max_corr_inds.shape